In [ ]:
#Imports
!pip install datasets
from datasets import load_dataset, DatasetDict # Import the load_dataset function
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

#Load Data

dataset_dict = load_dataset("shawhin/phishing-site-classification")

README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
# define pre-trained model path
model_path = "google-bert/bert-base-uncased"

# load model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# load model with binary classification head
id2label = {0: "not_phishing", 1: "phishing"}
label2id = {"not_phishing": 0, "phishing": 1}
model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=2, id2label=id2label, label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Freeze all base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [ ]:
# Data Pre-Processing

# define text preprocessing
def preprocess_function(examples):
  # return tokenized text with truncation
  return tokenizer(examples["text"], truncation=True)

# preprocess all datasets
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
# create data collator

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Define Evaluation Metrics

accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
  # get predictions
  predictions, labels = eval_pred

  # apply softmax to get probabilities
  predictions = np.exp(predictions) / np.exp(predictions).sum(axis=1, keepdims=True)

  # use probabilities of the postive class for ROC AUC
  positive_class_probs = predictions[:, 1]
  auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

  # predict most probable class
  predicted_classes = np.argmax(predictions, axis=1)

  # compute accuracy
  acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

  return {'Accuracy': acc, 'AUC': auc}

In [ ]:
# Training Parameters

# hyperparameters

lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [ ]:
# Fine tune Model

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()



<ipython-input-15-9ce9d1ecfc58>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kumarroushan-18 (kumarroushan-18-careerswipe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.505300,0.382973,0.816000,0.913000
2,0.409800,0.337012,0.844000,0.931000
3,0.359700,0.312252,0.853000,0.940000
4,0.357200,0.351782,0.847000,0.946000
5,0.351200,0.341680,0.862000,0.948000
6,0.347800,0.292300,0.869000,0.950000
7,0.335600,0.288071,0.876000,0.950000
8,0.312500,0.287983,0.869000,0.950000
9,0.313600,0.283730,0.871000,0.951000
10,0.314200,0.288482,0.864000,0.951000


TrainOutput(global_step=2630, training_loss=0.36068598816150044, metrics={'train_runtime': 198.8171, 'train_samples_per_second': 105.625, 'train_steps_per_second': 13.228, 'total_flos': 706603239165360.0, 'train_loss': 0.36068598816150044, 'epoch': 10.0})

In [ ]:
# Validation Data

# apply model to validation dataset
predictions = trainer.predict(tokenized_dataset["validation"])

# Extract the logits and labels from the predictions objects
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics functions
metrics = compute_metrics((logits, labels))

# Print the metrics
print(metrics)


{'Accuracy': np.float64(0.891), 'AUC': np.float64(0.945)}
